## Step-1: Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step-2: Download VisDrone DET dataset from Google Drive

In [5]:
!rm -r VisDrone

In [6]:
%%time
%%bash
if [ -d VisDrone]
then
  echo "data already exists"
else 
  mkdir -p VisDrone
  unzip -d VisDrone drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data/VisDrone2019-DET-val.zip
  unzip -d VisDrone/VisDrone2019-DET-test-dev drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data/VisDrone2019-DET-test-dev.zip
  unzip -d VisDrone drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data/VisDrone2019-DET-train.zip
  # unzip -d VisDrone drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data/VisDrone2019-DET-test-challenge.zip
fi

Archive:  drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data/VisDrone2019-DET-val.zip
   creating: VisDrone/VisDrone2019-DET-val/
  inflating: VisDrone/VisDrone2019-DET-val/.DS_Store  
   creating: VisDrone/VisDrone2019-DET-val/annotations/
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_02999_d_0000005.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_03499_d_0000006.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_03999_d_0000007.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_04527_d_0000008.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_05249_d_0000009.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_05499_d_0000010.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_05999_d_0000011.txt  
  inflating: VisDrone/VisDrone2019-DET-val/annotations/0000001_07999_d_0000012.txt  
  inflating: VisDrone/VisDrone2019-DET-val

bash: line 1: [: missing `]'


CPU times: user 1.29 s, sys: 269 ms, total: 1.56 s
Wall time: 26 s


## Step-3: Convert to Visdrone-YOLO format


In [7]:
import os
from pathlib import Path
import shutil
import cv2
import csv

In [8]:
def _convert_visidrone_DET_row_to_yolov7_row(size, box):
    dw = 1./size[1]
    dh = 1./size[0]
    x = int(box[0]) + int(box[2])/2.0
    y = int(box[1]) + int(box[3])/2.0
    w = int(box[2])
    h = int(box[3])
    return (box[5], x*dw,y*dh,w*dw,h*dh)

In [37]:
def _visdrone_DET_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-DET-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
        image_size_dict: dict
            contains the image_size info for various videos
    """
    # init paths/folders    
    input_ann_folder = str(Path(data_folder_dir) / "annotations")
    input_images_folder = str(Path(data_folder_dir) / "images")    
    # print(input_ann_folder)
    annotation_filepath_list = os.listdir(input_ann_folder)
    output_annotations_folder = str(Path(yolov7_output_folder_dir)/"labels")
    output_images_folder = str(Path(yolov7_output_folder_dir)/"images")
    shutil.copytree(input_images_folder, output_images_folder)
    Path(output_annotations_folder).mkdir(parents=True, exist_ok=True)
    annotation_dict = {}
    unique_image_sizes = set()
    for annotation_filename in annotation_filepath_list:
      yolov7_rows = []
      with open(Path(input_ann_folder)/annotation_filename) as file_obj:
        image_path = os.path.splitext(str(Path(data_folder_dir) / "images"/annotation_filename))[0]+".jpg"
        # print(image_path)
        img = cv2.imread(image_path)
        image_size = img.shape
        unique_image_sizes.add(image_size)
        # print("image size for (h,w,c) " + annotation_filename + " ==> " + str(image_size))
        reader_obj = csv.reader(file_obj)        
        for row in reader_obj:          
          yolov7_annotation_file_name = os.path.splitext(annotation_filename)[0] + "_" + str(row[0]).zfill(7) + ".txt" 
          yolov7_row = _convert_visidrone_DET_row_to_yolov7_row(image_size, row)
          yolov7_rows.append(yolov7_row)
      with open(str(Path(output_annotations_folder)/annotation_filename), "w", newline="") as f:
        for row in yolov7_rows:
          f.write(' '.join(str(item) for item in row)+'\n')
    for im_size in unique_image_sizes:
      print("image_size ==> " + str(im_size))

In [ ]:
! rm -r /content/VisDroneVideo

In [36]:
! rm -r /content/VisDrone/VisDrone2019-DET-YOLOv7

In [38]:
def _convert_visdrone_data_to_yolov7_format(data_folder_dir, yolov7_output_folder_dir):
  """
    Converts visdrone-VID annotations into yolov7 annotation.
    Args:
        data_folder_dir: str
            'VisDrone2019-DET-train' folder directory
        yolov7_output_folder_dir: str
            Output folder path for yolov7 converted data
  """
  _visdrone_DET_to_yolov7_annotation_files(data_folder_dir, yolov7_output_folder_dir)




In [39]:
def create_visdrone_DET_data_in_yolov7_format():
  _convert_visdrone_data_to_yolov7_format('/content/VisDrone/VisDrone2019-DET-train', '/content/VisDrone/VisDrone2019-DET-YOLOv7/train')
  _convert_visdrone_data_to_yolov7_format('/content/VisDrone/VisDrone2019-DET-val', '/content/VisDrone/VisDrone2019-DET-YOLOv7/val')
  _convert_visdrone_data_to_yolov7_format('/content/VisDrone/VisDrone2019-DET-test-dev', '/content/VisDrone/VisDrone2019-DET-YOLOv7/test-dev')
  ## Create data.yaml file
  try:
      with open('/content/VisDrone/VisDrone2019-DET-YOLOv7/data.yaml', 'w') as f:
          f.write('train: ../VisDrone/VisDrone2019-DET-YOLOv7/train/images\n')
          f.writelines('val: ../VisDrone/VisDrone2019-DET-YOLOv7/val/images\n')
          f.writelines('dev: ../VisDrone/VisDrone2019-DET-YOLOv7/test-dev/images\n')
          f.writelines('\n')
          f.writelines('nc: 12\n')
          f.writelines("names: ['ignore', 'pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor', 'others']\n")
  except FileNotFoundError:
      print("The 'docs' directory does not exist")

create_visdrone_DET_data_in_yolov7_format()

image_size ==> (756, 1344, 3)
image_size ==> (1080, 1920, 3)
image_size ==> (788, 1400, 3)
image_size ==> (1500, 2000, 3)
image_size ==> (765, 1360, 3)
image_size ==> (1048, 1398, 3)
image_size ==> (1050, 1400, 3)
image_size ==> (360, 480, 3)
image_size ==> (1042, 1389, 3)
image_size ==> (540, 960, 3)
image_size ==> (1078, 1916, 3)
image_size ==> (540, 960, 3)
image_size ==> (765, 1360, 3)
image_size ==> (1080, 1920, 3)
image_size ==> (1080, 1920, 3)
image_size ==> (788, 1400, 3)
image_size ==> (765, 1360, 3)
image_size ==> (1050, 1400, 3)
image_size ==> (540, 960, 3)
image_size ==> (1078, 1916, 3)


In [43]:
%%time
%%bash
zip -r 'VisDrone2019-DET-YOLOv7.zip' '/content/VisDrone/VisDrone2019-DET-YOLOv7'

Streaming output truncated to the last 5000 lines.
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/0000264_00801_d_0000198.txt (deflated 73%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/0000351_00197_d_0000520.txt (deflated 79%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/0000226_04475_d_0000017.txt (deflated 76%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/0000101_04436_d_0000021.txt (deflated 76%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/9999981_00000_d_0000122.txt (deflated 82%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/9999951_00000_d_0000163.txt (deflated 78%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/9999975_00000_d_0000031.txt (deflated 84%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/9999962_00000_d_0000005.txt (deflated 77%)
  adding: content/VisDrone/VisDrone2019-DET-YOLOv7/train/labels/9999940_00000_d_0000083.txt (

In [57]:
!cp "/content/VisDrone2019-DET-YOLOv7.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/DET_data"

In [45]:
!git clone https://github.com/dnautiyal/MLOps4-Capstone.git

Cloning into 'MLOps4-Capstone'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 65 (delta 20), reused 52 (delta 15), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [52]:
!python /content/MLOps4-Capstone/src/yolo_data_utils/yolo_draw_annotation_box.py

0000277_03601_d_0000556
Box number: 66 Image number: 1
0000269_00201_d_0000349
Box number: 120 Image number: 2
0000291_05001_d_0000892
Box number: 257 Image number: 3
0000277_00001_d_0000539
Box number: 359 Image number: 4
0000323_02601_d_0000645
Box number: 453 Image number: 5
0000026_04500_d_0000033
Box number: 500 Image number: 6
0000280_03201_d_0000628
Box number: 583 Image number: 7
0000360_00393_d_0000715
Box number: 662 Image number: 8
0000269_00001_d_0000348
Box number: 794 Image number: 9
0000023_00000_d_0000008
Box number: 834 Image number: 10
0000327_04201_d_0000732
Box number: 945 Image number: 11
0000330_03001_d_0000815
Box number: 990 Image number: 12
0000327_01801_d_0000720
Box number: 1052 Image number: 13
0000163_00001_d_0000001
Box number: 1124 Image number: 14
0000289_07201_d_0000847
Box number: 1164 Image number: 15
0000280_02201_d_0000623
Box number: 1252 Image number: 16
0000242_02762_d_0000010
Box number: 1285 Image number: 17
0000103_03349_d_0000031
Box number: 

In [ ]:
%%time
%%bash
zip -r 'VisDrone2019-VID-YOLOv7-AnnotatedTest.zip' '/content/save_image'

In [ ]:
!cp "/content/VisDrone2019-VID-YOLOv7-AnnotatedTest.zip" "/content/drive/MyDrive/Deepak/FourthBrain/MLOps_Course/CapstoneProject/TrainYOLOv7/data/video_data/YOLOv7"